# IREI: Search Engines & Real World Data
### Víctor Morcuende Castell and Guillermo Nájera Lavid
#### Course 2022-2023

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import ndcg_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# Load data
df = pd.read_excel('loinc_dataset-v2.xlsx')
target = df.pop('relevant')

In [ ]:
# We eliminate irrelevant information
df.pop('comments')

In [ ]:
# Using OneHotEncoder
ohe = preprocessing.OneHotEncoder(sparse=False)
df = pd.DataFrame(ohe.fit_transform(df), 
                          columns=ohe.get_feature_names_out(df.columns.tolist()),
                          index=df.index)

In [2]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.2)

In [3]:
# Train AdaRank model
def train_adarank(X_train, y_train):
    base_estimator = DecisionTreeRegressor(max_depth=1)
    model = AdaBoostRegressor(base_estimator=base_estimator, n_estimators=100, loss='linear')
    model.fit(X_train, y_train)
    return model

In [4]:
# Evaluate AdaRank model
def evaluate_adarank(model, X_test, y_test):
    y_pred = model.predict(X_test)
    ndcg = ndcg_score([y_test], [y_pred], k=X_test.shape[1])
    return ndcg

In [5]:
model = train_adarank(X_train, y_train)
model_ndcg = evaluate_adarank(model, X_test, y_test)
print('Model NDCG score:', model_ndcg)

/Users/victor/miniconda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


ValueError: could not convert string to float: '2039-6'